# Quickstart - Avatarization with parameters

## Connection

In [ ]:
# This is the client that you'll be using for all of your requests
from avatars.client import ApiClient
from avatars.models import AvatarizationJobCreate, AvatarizationParameters
from avatars.models import ReportCreate
from avatars.models import PrivacyMetricsJobCreate, PrivacyMetricsParameters
from avatars.models import SignalMetricsJobCreate, SignalMetricsParameters

# The following are not necessary to run avatar but are used in this tutorial
import pandas as pd
import io
import os

url = os.environ.get("AVATAR_BASE_URL")
username = os.environ.get("AVATAR_USERNAME")
password = os.environ.get("AVATAR_PASSWORD")
print(url)

In [ ]:
# Change this to your actual server endpoint, e.g. base_url="https://avatar.company.com"
client = ApiClient(base_url=url)
client.authenticate(username=username, password=password)

## Loading data

In [ ]:
df = pd.read_csv("../fixtures/iris.csv")
dataset = client.pandas_integration.upload_dataframe(df)

## Analyze your data

In [ ]:
while dataset.summary is None:
    dataset = client.datasets.analyze_dataset(dataset.id)

In [ ]:
for var in dataset.summary.stats:
    print("---------")
    for stat in var:
        print(stat)

## Creating and launching an avatarization job and metrics

In [ ]:
avatarization_job = client.jobs.create_avatarization_job(
    AvatarizationJobCreate(
        parameters=AvatarizationParameters(k=20, dataset_id=dataset.id),
    )
)

avatarization_job = client.jobs.get_avatarization_job(avatarization_job.id, timeout=10)
print(avatarization_job.status)
print(avatarization_job.result)  # there is no metrics

## Calculate Privacy Metrics

In [ ]:
privacy_job = client.jobs.create_privacy_metrics_job(
    PrivacyMetricsJobCreate(
        parameters=PrivacyMetricsParameters(
            original_id=dataset.id,
            unshuffled_avatars_id=avatarization_job.result.sensitive_unshuffled_avatars_datasets.id,
            closest_rate_percentage_threshold=0.3,
            closest_rate_ratio_threshold=0.3,
            known_variables=[
                "sepal.length",
                "petal.length",
            ],
            target="variety"
        ),
    )
)

privacy_job = client.jobs.get_privacy_metrics(privacy_job.id, timeout=10)

print(privacy_job.status)
print("*** Privacy metrics ***")
for metric in privacy_job.result:
    print(metric)

## Calculate Utility Metrics

In [ ]:
signal_job = client.jobs.create_signal_metrics_job(
    SignalMetricsJobCreate(
        parameters=SignalMetricsParameters(
            original_id=dataset.id,
            avatars_id=avatarization_job.result.avatars_dataset.id
        ),
    )
)

signal_job = client.jobs.get_signal_metrics(signal_job.id, timeout=10)
print(signal_job.status)
print("*** Utility metrics ***")
for metric in signal_job.result:
    print(metric)

## Retrieve avatars

In [ ]:
# Download the avatars as a string
avatars_str = client.datasets.download_dataset(avatarization_job.result.avatars_dataset.id)

# Download the avatars as a pandas dataframe
avatars_df = client.pandas_integration.download_dataframe(avatarization_job.result.avatars_dataset.id)

## Retrieving the avatarization report

In [ ]:
report = client.reports.create_report(
    ReportCreate(
        avatarization_job_id=avatarization_job.id,
        privacy_job_id=privacy_job.id,
        signal_job_id=signal_job.id,
    ),
    timeout=10,
)
result = client.reports.download_report(id=report.id)

with open("./my_avatarization_report.pdf", "wb") as f:
    f.write(result)

The report is now generated and available on your machine